# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mercedes1996/geogradev/blob/main/docs/examples/gee.ipynb)

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

In [ ]:
ee.Initialize(project="ee-aguirrecername")  # Replace with your GEE project name

In [ ]:
m = geemap.Map()
m

In [ ]:
# Initialize Earth Engine
ee.Initialize(project="ee-aguirrecername")
m = geemap.Map()
m.centerObject(
    ee.Geometry.Point([-76.5, -6.8]), 8
)  # center the map on a specific point
m

In [ ]:
# question 1
# Initialize Earth Engine and map
ee.Initialize(project="ee-aguirrecername")  # Reemplaza con tu proyecto
m = geemap.Map()
m.centerObject(ee.Geometry.Point([-76.5, -6.8]), 8)  # Centrar en San Martín, Perú

# Create a square of 200 km on each side around the center point
point = ee.Geometry.Point([-76.5, -6.8])
side_length = 200000  # 200 km en metros
half_side = side_length / 2

# Coordinates of the rectangle (west, south, east, north)
rectangle = ee.Geometry.Rectangle(
    [
        -76.5 - (half_side / 111320),
        -6.8 - (half_side / 111320),
        -76.5 + (half_side / 111320),
        -6.8 + (half_side / 111320),
    ]
)

# Load DEM (SRTM) and trim square
dem = ee.Image("USGS/SRTMGL1_003").clip(rectangle)

# Add to map with elevation palette
palette = ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"]
m.addLayer(
    dem, {"min": 100, "max": 4000, "palette": palette}, "DEM San Martín (200x200 km)"
)
m.addLayer(
    rectangle, {"color": "red"}, "Área de Estudio (Cuadrado)"
)  # Opcional: mostrar el cuadrado

m

In [ ]:
# Question 2
# Initialize Earth Engine and map
ee.Initialize(project="ee-aguirrecername")
m = geemap.Map()
m.centerObject(ee.Geometry.Point([-76.5, -6.8]), 8)

# 1. Define the square area of ​​200x200 km (same as in Exercise 1)
point = ee.Geometry.Point([-76.5, -6.8])
side_length = 200000
half_side = side_length / 2
rectangle = ee.Geometry.Rectangle(
    [
        -76.5 - (half_side / 111320),
        -6.8 - (half_side / 111320),
        -76.5 + (half_side / 111320),
        -6.8 + (half_side / 111320),
    ]
)

# 2. Load Sentinel-2, filter by area, date and cloud cover (2023)
s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(rectangle)
    .filterDate("2023-01-01", "2023-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
)

# 3. Create medium composite (to reduce noise) and trim
composite = s2.median().clip(rectangle)

# 4. Display with natural bands (RGB) and false color (Infrared)
vis_natural = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 2500, "gamma": 1.2}

vis_false_color = {"bands": ["B8", "B4", "B3"], "min": 0, "max": 3000, "gamma": 1.2}

# Add layers to the map
m.addLayer(composite, vis_natural, "Sentinel-2 RGB Natural (2023)")
m.addLayer(composite, vis_false_color, "Sentinel-2 false Color (2023)")
m.addLayer(rectangle, {"color": "red"}, "Área de Estudio", False)  # Opcional

# Add layer controls
m.addLayerControl()
m

In [ ]:
# Question 3
# Initialize Earth Engine
ee.Initialize()
m = geemap.Map()

# 1. Get Miami-Dade County boundary
county = (
    ee.FeatureCollection("TIGER/2018/Counties")
    .filter(ee.Filter.eq("NAME", "Miami-Dade"))
    .filter(ee.Filter.eq("STATEFP", "12"))
    .first()
)

# 2. Load NAIP imagery (most recent year)
naip = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(county.geometry())
    .filterDate("2020-01-01", "2023-12-31")
    .mosaic()
    .clip(county)
)

# 3. Display the map
m.addLayer(naip, {"bands": ["R", "G", "B"]}, "NAIP Miami-Dade")
m.centerObject(county, 10)
m

In [ ]:
# Question 4
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()
m = geemap.Map()

# 1. Load Florida's boundary
florida = (
    ee.FeatureCollection("TIGER/2018/States")
    .filter(ee.Filter.eq("NAME", "Florida"))
    .first()
)

# 2. Use the NEW watershed dataset (HydroBASINS)
watersheds = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_8").filterBounds(
    florida.geometry()
)  # Only watersheds in Florida

# 3. Style: Blue outlines, no fill
style = {
    "color": "blue",  # Outline color
    "fillColor": "00000000",  # Transparent fill
    "width": 1,  # Line thickness
}

# 4. Add to map
m.addLayer(florida, {"color": "red"}, "Florida Boundary")  # Optional
m.addLayer(watersheds.style(**style), {}, "Watersheds (HydroBASINS)")
m.centerObject(florida, 6)  # Zoom to Florida
m

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
# Question5
# Initialize Earth Engine
ee.Initialize()
m = geemap.Map()

# 1. Load Florida boundary
florida = (
    ee.FeatureCollection("TIGER/2018/States")
    .filter(ee.Filter.eq("NAME", "Florida"))
    .first()
)

# 2. Load CORRECT NLCD datasets (2021 release paths)
nlcd_2001 = (
    ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2001")
    .select("landcover")
    .clip(florida.geometry())
)
nlcd_2021 = (
    ee.Image("USGS/NLCD_RELEASES/2021_REL/NLCD/2021")
    .select("landcover")
    .clip(florida.geometry())
)
# 3. NLCD color palette (official colors)
nlcd_palette = [
    "466b9f",  # Water
    "d1def8",  # Ice/Snow
    "dec5c5",  # Developed
    "d99282",
    "eb0000",
    "ab0000",
    "b3ac9f",  # Barren
    "68ab5f",  # Forest
    "1c5f2c",
    "b5c58f",  # Shrub/Scrub
    "ccb879",  # Grassland
    "dfdfc2",  # Pasture
    "d1d182",  # Crops
    "a3cc51",  # Wetlands
    "82ba9e",
    "dcd939",
    "ab6c28",
]

# 4. Create split-panel map
left_layer = geemap.ee_tile_layer(
    nlcd_2001, {"min": 0, "max": 95, "palette": nlcd_palette}, "NLCD 2001"
)
right_layer = geemap.ee_tile_layer(
    nlcd_2021, {"min": 0, "max": 95, "palette": nlcd_palette}, "NLCD 2021"
)

m.split_map(left_layer, right_layer)
m.centerObject(florida, 6)

# 5. Add simplified NLCD legend
legend_dict = {
    "Water": "466b9f",
    "Developed": "eb0000",
    "Forest": "1c5f2c",
    "Shrubland": "b5c58f",
    "Grassland": "dfdfc2",
    "Agriculture": "d1d182",
    "Wetlands": "a3cc51",
    "Barren": "b3ac9f",
}

m.add_legend(legend_dict=legend_dict, title="NLCD Land Cover")
m

In [ ]:
# Question 6
# Initialize Earth Engine
ee.Initialize()

# Define San Martín area (200x200 km)
west = -76.5 - (100 / 111.32)
east = -76.5 + (100 / 111.32)
south = -6.8 - (100 / 111.32)
north = -6.8 + (100 / 111.32)

roi = ee.Geometry.BBox(west, south, east, north)

# Generate NDVI timelapse
timelapse = geemap.landsat_timelapse(
    roi=roi,
    out_gif="san_martin.gif",
    start_year=1984,
    end_year=2023,
    bands=["NIR", "Red", "Green"],
    frames_per_second=5,
    title="San Martín Vegetation (1984-2023)",
    font_color="white",
    progress_bar_color="blue",
)

# Display
geemap.show_image(timelapse)